In [8]:
import numpy as np
import glob
import dill
from tqdm import tqdm
from utils import MinMaxScaler, StandardScaler, make_dir, sort_fn, save_object
import matplotlib.pyplot as plt
from multiprocessing import Pool

In [9]:
x_paths = sorted(glob.glob("../data/SPCAM5/inputs_*"), key=sort_fn)
y_paths = sorted(glob.glob("../data/SPCAM5/outputs_*"), key=sort_fn)

In [25]:
n = 183 # first dataset: 6 monnths
# train_x = x_paths[:int(n*0.8)]
# train_y = y_paths[:int(n*0.8)]
# val_x = x_paths[int(n*0.8): n]
# val_y = y_paths[int(n*0.8): n]
# test_x = x_paths[n:365]
# test_y = y_paths[n:365]

n = 365 # second dataset: 1 year
train_x = x_paths[:int(n*0.8)]
train_y = y_paths[:int(n*0.8)]
val_x = x_paths[int(n*0.8): n]
val_y = y_paths[int(n*0.8): n]
test_x = x_paths[n:]
test_y = y_paths[n:]

In [26]:
x_scaler_minmax = MinMaxScaler()
y_scaler_minmax = MinMaxScaler()
x_scaler_standard = StandardScaler()
y_scaler_standard = StandardScaler()

x_max = 0
y_max = 0

def fit_scalers(paths, x_max, y_max):
    x_path, y_path = paths
    x = np.load(x_path, mmap_mode="r")
    y = np.load(y_path, mmap_mode="r")[:, :26]
    x_scaler_minmax.fit(x)
    y_scaler_minmax.fit(y)
    x_scaler_standard.fit(x)
    y_scaler_standard.fit(y)
    return np.maximum(x_max, np.max(np.abs(x), 0)), np.maximum(y_max, np.max(np.abs(y), 0))


for i in tqdm(range(len(train_x)), total=len(train_x)):
    if i == 146:
        np.save(f"dataset_1_x_max.npy", x_max)
        np.save(f"dataset_1_y_max.npy", y_max)
        save_object(x_scaler_minmax, f"dataset_1_x_scaler_minmax.pkl")
        save_object(y_scaler_minmax, f"dataset_1_y_scaler_minmax.pkl")
        save_object(x_scaler_standard, f"dataset_1_x_scaler_standard.pkl")
        save_object(y_scaler_standard, f"dataset_1_y_scaler_standard.pkl")
    x_max, y_max = fit_scalers([train_x[i], train_y[i]], x_max, y_max)

np.save(f"dataset_2_x_max.npy", x_max)
np.save(f"dataset_2_y_max.npy", y_max)

save_object(x_scaler_minmax, f"dataset_2_x_scaler_minmax.pkl")
save_object(y_scaler_minmax, f"dataset_2_y_scaler_minmax.pkl")
save_object(x_scaler_standard, f"dataset_2_x_scaler_standard.pkl")
save_object(y_scaler_standard, f"dataset_2_y_scaler_standard.pkl")

100%|██████████| 292/292 [33:06<00:00,  6.80s/it]


In [ ]:
# x_scaler_minmax = dill.load(open("dataset_1_x_scaler_minmax.pkl", "rb"))
# y_scaler_minmax = dill.load(open("dataset_1_y_scaler_minmax.pkl", "rb"))
# x_scaler_standard = dill.load(open("dataset_1_x_scaler_standard.pkl", "rb"))
# y_scaler_standard = dill.load(open("dataset_1_y_scaler_standard.pkl", "rb"))